In [52]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [53]:
count=0
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)

# len4_ndb is left joined so delete duplicated rows
data=data.drop_duplicates(subset=["id","year","総計"])
# drop never treated group
data=data.dropna(subset=["release_year"])
data=data.dropna(subset=["総計"])
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_l4,Approved,KEGG,Ingredient,date,release_year,成分,variable,id,revenue
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,3.261297e+08
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,2.377048e+08
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,6.583945e+08
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,6.363609e+08
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,3.368936e+08


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [54]:
data["elapsed"]=data["year"]-data["release_year"]
#　不要なところを除外
# data.loc[(data['elapsed'] > 10) | (data["elapsed"]< -5), 'elapsed'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elapsed"],prefix="elapsed")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","薬効分類"])[sumcolumns].sum().reset_index()
# save temporarily
dummy_cols=dummies.columns.to_list()

# sum by generic
sum_generic=data.groupby(["year","薬効分類","後発品区分"])[sumcolumns].sum().reset_index()

if count:
    # count ver.
    aggregated_df = pd.concat([
        data.groupby(['year', '薬効分類']).first().reset_index()[['year', '薬効分類']],
        dummies.groupby([data['year'], data['薬効分類']]).sum().reset_index(drop=True)
    ], axis=1).reset_index(drop=True)
else:
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', '薬効分類']).first().reset_index()[['year', '薬効分類']],
        dummies.groupby([data['year'], data['薬効分類']]).apply(lambda x: (x[dummy_cols] > 0).any().astype(int)).reset_index(drop=True)
    ], axis=1).reset_index(drop=True)
# # sumをとったところを追加する

aggregated_df[sumcolumns]=sums[sumcolumns]
aggregated_df.rename(columns={"総計":"sum_quantity","revenue":"sum_revenue"},inplace=True)
aggregated_df["mean_price"]=aggregated_df["sum_revenue"]/aggregated_df["sum_quantity"]
aggregated_df["lag_mean_price"]=aggregated_df.groupby(["薬効分類"])["mean_price"].shift(1)
# lag sum_quantity
aggregated_df["lag_sum_quantity"]=aggregated_df.groupby(["薬効分類"])["sum_quantity"].shift(1)
aggregated_df.head()

,year,薬効分類,elapsed_m6,elapsed_m5,elapsed_m4,elapsed_m3,elapsed_m2,elapsed_m1,elapsed_0,elapsed_1,...,elapsed_27,elapsed_28,elapsed_29,elapsed_30,elapsed_31,sum_quantity,sum_revenue,mean_price,lag_mean_price,lag_sum_quantity
0,2014,114.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3.429951e+09,1.094683e+11,31.915396,NaN,NaN
1,2014,121.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.840017e+07,3.186312e+08,6.583267,NaN,NaN
2,2014,123.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.596449e+08,2.033818e+09,12.739638,NaN,NaN
3,2014,131.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,8.145575e+08,1.297159e+11,159.247093,NaN,NaN
4,2014,132.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.790076e+08,3.798759e+10,212.212151,NaN,NaN


In [55]:
## genericとbrandのそれぞれの処方量と売上を集計
sum_generic["後発品区分"]=sum_generic["後発品区分"].astype("int")
sum_generic=sum_generic.pivot_table(index=["year","薬効分類"],columns="後発品区分",values=sumcolumns,aggfunc="sum").reset_index()
sum_generic

year   薬効分類       revenue                          総計              
後発品区分                          0             1             0             1
0      2014  114.0  1.035691e+11  5.899116e+09  2.619996e+09  8.099551e+08
1      2014  121.0  3.016000e+08  1.703117e+07  4.539382e+07  3.006345e+06
2      2014  123.0  1.903159e+09  1.306591e+08  1.390288e+08  2.061612e+07
3      2014  131.0  1.192776e+11  1.043834e+10  7.030695e+08  1.114880e+08
4      2014  132.0  3.576847e+10  2.219123e+09  1.742795e+08  4.728111e+06
..      ...    ...           ...           ...           ...           ...
195    2021  441.0  8.816861e+08  3.153723e+08  1.060883e+08  5.532848e+07
196    2021  449.0  1.125259e+11  6.424882e+10  1.800630e+09  2.793408e+09
197    2021  625.0  1.429809e+11  7.867905e+09  7.157256e+07  7.232269e+07
198    2021  629.0  4.585181e+10  3.655754e+09  1.164413e+08  4.626545e+07
199    2021  799.0  7.123352e+09  2.065872e+08  4.594945e+07  3.683486e+05

[200 rows x 6 columns]

In [56]:
print(sum_generic.columns)
sum_generic.columns = [f"{col[0]}{col[1]}" for col in sum_generic.columns]
sum_generic.columns

MultiIndex([(   'year', ''),
            (   '薬効分類', ''),
            ('revenue',  0),
            ('revenue',  1),
            (     '総計',  0),
            (     '総計',  1)],
           names=[None, '後発品区分'])


Index(['year', '薬効分類', 'revenue0', 'revenue1', '総計0', '総計1'], dtype='object')

In [57]:
aggregated_df[sum_generic.columns]=sum_generic[sum_generic.columns]

In [58]:
aggregated_df

,year,薬効分類,elapsed_m6,elapsed_m5,elapsed_m4,elapsed_m3,elapsed_m2,elapsed_m1,elapsed_0,elapsed_1,...,elapsed_31,sum_quantity,sum_revenue,mean_price,lag_mean_price,lag_sum_quantity,revenue0,revenue1,総計0,総計1
0,2014,114.0,0,0,0,0,0,0,0,0,...,0,3.429951e+09,1.094683e+11,31.915396,NaN,NaN,1.035691e+11,5.899116e+09,2.619996e+09,8.099551e+08
1,2014,121.0,0,0,0,0,0,0,0,0,...,0,4.840017e+07,3.186312e+08,6.583267,NaN,NaN,3.016000e+08,1.703117e+07,4.539382e+07,3.006345e+06
2,2014,123.0,0,0,0,0,0,0,0,0,...,0,1.596449e+08,2.033818e+09,12.739638,NaN,NaN,1.903159e+09,1.306591e+08,1.390288e+08,2.061612e+07
3,2014,131.0,0,0,0,0,0,0,0,0,...,0,8.145575e+08,1.297159e+11,159.247093,NaN,NaN,1.192776e+11,1.043834e+10,7.030695e+08,1.114880e+08
4,2014,132.0,0,0,0,0,0,0,0,0,...,0,1.790076e+08,3.798759e+10,212.212151,NaN,NaN,3.576847e+10,2.219123e+09,1.742795e+08,4.728111e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2021,441.0,0,0,0,0,0,0,0,0,...,0,1.614168e+08,1.197058e+09,7.415946,7.430559,1.606074e+08,8.816861e+08,3.153723e+08,1.060883e+08,5.532848e+07
196,2021,449.0,0,0,0,0,0,0,0,0,...,0,4.594037e+09,1.767747e+11,38.479161,41.530246,4.391606e+09,1.125259e+11,6.424882e+10,1.800630e+09,2.793408e+09
197,2021,625.0,0,0,0,0,0,0,0,0,...,0,1.438952e+08,1.508488e+11,1048.324019,1107.139880,1.414130e+08,1.429809e+11,7.867905e+09,7.157256e+07,7.232269e+07
198,2021,629.0,0,0,0,0,0,0,0,0,...,0,1.627067e+08,4.950757e+10,304.274861,304.806118,1.592574e+08,4.585181e+10,3.655754e+09,1.164413e+08,4.626545e+07


In [59]:
# sums

In [60]:
yakuji=pd.read_csv("../merged/len3_ship_agg_drop_never.csv",encoding="shift-jis",index_col=0)
yakuji=yakuji[["year","id","rx_dom","otc_dom","stock_rx","stock_otc","stock_self"]]
aggregated_df=aggregated_df.merge(yakuji,left_on=["year","薬効分類"],right_on=["year","id"],how="left")
aggregated_df.head()

,year,薬効分類,elapsed_m6,elapsed_m5,elapsed_m4,elapsed_m3,elapsed_m2,elapsed_m1,elapsed_0,elapsed_1,...,revenue0,revenue1,総計0,総計1,id,rx_dom,otc_dom,stock_rx,stock_otc,stock_self
0,2014,114.0,0,0,0,0,0,0,0,0,...,1.035691e+11,5.899116e+09,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
1,2014,121.0,0,0,0,0,0,0,0,0,...,3.016000e+08,1.703117e+07,4.539382e+07,3.006345e+06,121,12343814,0.0,1340818.0,0.0,0.0
2,2014,123.0,0,0,0,0,0,0,0,0,...,1.903159e+09,1.306591e+08,1.390288e+08,2.061612e+07,123,2463354,0.0,181986.0,0.0,0.0
3,2014,131.0,0,0,0,0,0,0,0,0,...,1.192776e+11,1.043834e+10,7.030695e+08,1.114880e+08,131,286216591,59438256.0,24420274.0,13291858.0,0.0
4,2014,132.0,0,0,0,0,0,0,0,0,...,3.576847e+10,2.219123e+09,1.742795e+08,4.728111e+06,132,40223232,19292767.0,4853936.0,5954843.0,0.0


In [61]:
merged_data=pd.merge(data,aggregated_df,on=["year","薬効分類"],how="left")

In [62]:
merged_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,revenue0,revenue1,総計0,総計1,id_y,rx_dom,otc_dom,stock_rx,stock_otc,stock_self
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,1.035691e+11,5.899116e+09,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,1.035691e+11,5.899116e+09,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,1.035691e+11,5.899116e+09,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,1.035691e+11,5.899116e+09,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,1.035691e+11,5.899116e+09,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0


connect generic usage

In [63]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [64]:
merged_df=merged_data.merge(dusage,left_on="year",right_on="Year",how="left")
merged_df.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,総計0,総計1,id_y,rx_dom,otc_dom,stock_rx,stock_otc,stock_self,Year,Percentage
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,2.619996e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55


In [65]:
id_dummies=pd.get_dummies(merged_df["薬効分類"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
# share of the product
df_with_dummies["q_share"]=df_with_dummies["総計"]/df_with_dummies["sum_quantity"]
df_with_dummies["r_share"]=df_with_dummies["revenue"]/(df_with_dummies["sum_revenue"]+df_with_dummies["otc_dom"]*1000)

# aggregated generic share in market i (id len 3)
df_with_dummies["generic_share_q"]=df_with_dummies["総計1"]/df_with_dummies["sum_quantity"]
df_with_dummies["generic_share_r"]=df_with_dummies["revenue1"]/df_with_dummies["sum_revenue"]

df_with_dummies["mean_price_g"]=df_with_dummies["revenue1"]/df_with_dummies["総計1"]
df_with_dummies["mean_price_b"]=df_with_dummies["revenue0"]/df_with_dummies["総計0"]

df_with_dummies.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r,mean_price_g,mean_price_b
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,0,0,0,0,0.007258,0.002068,0.236142,0.053889,7.283263,39.530262
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,0,0,0,0,0.002987,0.001507,0.236142,0.053889,7.283263,39.530262
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,0,0,0,0,0.014653,0.004175,0.236142,0.053889,7.283263,39.530262
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,0,0,0,0,0.007997,0.004036,0.236142,0.053889,7.283263,39.530262
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,0,0,0,0,0.001557,0.002136,0.236142,0.053889,7.283263,39.530262


In [66]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
if not(count):
    # dummy ver
    df_with_dummies.to_csv("../merged/len3_ndb_agg_blp_DropNever.csv",encoding="shift-jis")
else:
    # count ver
    df_with_dummies.to_csv("../merged/len4_ndb_sum_agg_blp_DropNever.csv",encoding="shift-jis")

In [67]:
df_with_dummies.shape

(25798, 108)

In [68]:
df_price=df_with_dummies.drop_duplicates(subset=["year","薬効分類"]).reset_index(drop=1)[["year","薬効分類","mean_price","mean_price_g","mean_price_b"]]
df_price.to_csv("../merged/len3_ndb_mean_prices.csv",encoding="shift-jis")